In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from cmaqpy import prepemis

In [52]:
# Read in the generator data previously preprocessed by ERTAC EGU tool

base_df = pd.read_csv('test_calc_hourly_base.csv', 
    dtype={'ertac_region': 'object',
    'ertac_fuel_unit_type_bin': 'object',
    'state': 'object',
    'facility_name': 'object',
    'orispl_code': 'int64',
    'unitid': 'object',
    'op_date': 'object',
    'op_hour': 'str',
    'op_time': 'float64',
    'gload (MW-hr)': 'object',
    'sload (1000 lbs)': 'float64',
    'so2_mass (lbs)': 'object',
    'so2_mass_measure_flg': 'object',
    'so2_rate (lbs/mmBtu)': 'float64',
    'so2_rate_measure_flg': 'object',
    'nox_rate (lbs/mmBtu)': 'float64',
    'nox_rate_measure_flg': 'object',
    'nox_mass (lbs)': 'object',
    'nox_mass_measure_flg': 'object',
    'co2_mass (tons)': 'object',
    'co2_mass_measure_flg': 'object',
    'co2_rate (tons/mmBtu)': 'float64',
    'co2_rate_measure_flg': 'object',
    'heat_input (mmBtu)': 'float64'})

# Change the orispl_code to a string
base_df = base_df.astype({'orispl_code': 'str'})
# Pad the string for formatting
base_df['op_hour'] = base_df['op_hour'].str.zfill(2)
# Add a datetime column 
base_df['datetime'] = pd.to_datetime(base_df['op_date'] + ' ' + base_df['op_hour'])
# Add the datetime as an additional index
base_df.head()

,ertac_region,ertac_fuel_unit_type_bin,state,facility_name,orispl_code,unitid,op_date,op_hour,op_time,gload (MW-hr),...,nox_rate (lbs/mmBtu),nox_rate_measure_flg,nox_mass (lbs),nox_mass_measure_flg,co2_mass (tons),co2_mass_measure_flg,co2_rate (tons/mmBtu),co2_rate_measure_flg,heat_input (mmBtu),datetime
0,SRSE,Boiler Gas,AL,Barry,3,1,2016-01-01,00,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-01-01 00:00:00
1,SRSE,Boiler Gas,AL,Barry,3,1,2016-01-01,01,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-01-01 01:00:00
2,SRSE,Boiler Gas,AL,Barry,3,1,2016-01-01,02,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-01-01 02:00:00
3,SRSE,Boiler Gas,AL,Barry,3,1,2016-01-01,03,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-01-01 03:00:00
4,SRSE,Boiler Gas,AL,Barry,3,1,2016-01-01,04,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-01-01 04:00:00


In [53]:
# Read in ML CO2 emissions estimations
ml_co2 = prepemis.fmt_like_camd(data_file='../ny_emis/ml_output/pred_xg_co2.csv', lu_file='../ny_emis/ed_output/RGGI_to_NYISO.csv')

In [54]:
# Read in ML NOx emissions estimations
ml_nox = prepemis.fmt_like_camd(data_file='../ny_emis/ml_output/pred_xg_nox.csv', lu_file='../ny_emis/ed_output/RGGI_to_NYISO.csv')

In [55]:
# Read in ML SO2 emissions estimations
ml_so2 = prepemis.fmt_like_camd(data_file='../ny_emis/ml_output/pred_xg_so2.csv', lu_file='../ny_emis/ed_output/RGGI_to_NYISO.csv')

In [56]:
# Read in NY Simple Net generation
ed_gen = prepemis.fmt_like_camd(data_file='../ny_emis/ed_output/thermal_with_renewable_20160805_20160815.csv', lu_file='../ny_emis/ed_output/RGGI_to_NYISO.csv')

Now, we want to replace the emissions values that we got from the ML model in the CAMD database. We use the follow steps
1. Match units to a ORISPL and UNIT ID (done in prep)
2. Check to see how many of the units in the Simple Net are in the CAMD data (some will be too small) (done in prep)
3. Add additional rows to the Simple Net data when the NYISO PTID corresponds to multiple UNIT IDs and divide the generation evenly among the units (done in prep)
4. Replace the data in `calc_hourly_base.csv`

In [57]:
idx = 62
# Get the ORISPL and the Unit ID
egu_orispl = ml_co2.loc[idx].ORISPL
egu_unitid = ml_co2.loc[idx]['Unit ID']
print(f'ORISPL: {egu_orispl}\tUNIT ID:{egu_unitid}')

ORISPL: 2535	UNIT ID:1


In [58]:
ml_co2.columns[5]

Timestamp('2016-08-05 00:00:00')

In [59]:
# Extract this ORISPL and UNIT ID from the base DataFrame
egu_df = base_df.loc[(base_df['orispl_code'] == egu_orispl) & (base_df['unitid'] == egu_unitid)]
# Extract the correct time window
egu_df = egu_df.loc[base_df['datetime'].isin(ml_co2.columns[5:])]
egu_df.head()

,ertac_region,ertac_fuel_unit_type_bin,state,facility_name,orispl_code,unitid,op_date,op_hour,op_time,gload (MW-hr),...,nox_rate (lbs/mmBtu),nox_rate_measure_flg,nox_mass (lbs),nox_mass_measure_flg,co2_mass (tons),co2_mass_measure_flg,co2_rate (tons/mmBtu),co2_rate_measure_flg,heat_input (mmBtu),datetime
846167,NYUP,Coal,NY,"Cayuga Operating Company, LLC",2535,1,2016-08-05,00,1.0,159.0,...,0.075,Calculated,112.7,Measured,154.2,Measured,0.103,Calculated,1502.6,2016-08-05 00:00:00
846168,NYUP,Coal,NY,"Cayuga Operating Company, LLC",2535,1,2016-08-05,01,1.0,159.0,...,0.077,Calculated,115.8,Measured,154.4,Measured,0.103,Calculated,1504.5,2016-08-05 01:00:00
846169,NYUP,Coal,NY,"Cayuga Operating Company, LLC",2535,1,2016-08-05,02,1.0,156.0,...,0.071,Calculated,105.5,Measured,152.4,Measured,0.103,Calculated,1485.5,2016-08-05 02:00:00
846170,NYUP,Coal,NY,"Cayuga Operating Company, LLC",2535,1,2016-08-05,03,1.0,156.0,...,0.072,Calculated,106.5,Measured,151.8,Measured,0.103,Calculated,1479.6,2016-08-05 03:00:00
846171,NYUP,Coal,NY,"Cayuga Operating Company, LLC",2535,1,2016-08-05,04,1.0,157.0,...,0.073,Calculated,108.2,Measured,152.0,Measured,0.103,Calculated,1481.8,2016-08-05 04:00:00


In [60]:
ml_co2.loc[idx, ml_co2.columns[5:]]

2016-08-05 00:00:00     51.75686
2016-08-05 01:00:00     51.75686
2016-08-05 02:00:00     51.75686
2016-08-05 03:00:00     51.75686
2016-08-05 04:00:00     51.75686
                         ...    
2016-08-15 19:00:00    150.41125
2016-08-15 20:00:00    150.41125
2016-08-15 21:00:00    150.41125
2016-08-15 22:00:00    150.41125
2016-08-15 23:00:00     51.75686
Name: 62, Length: 264, dtype: object

In [61]:
# Replace the CO2 emissions values
# NOTE: this is probably a dangerous way of doing this -- might be better to add the datetime as another index in the egu_df
egu_df['co2_mass (tons)'] = ml_co2.loc[idx, ml_co2.columns[5:]].values
# Replace the SO2 emissions values
egu_df['so2_mass (lbs)'] = ml_so2.loc[idx, ml_so2.columns[5:]].values
# Replace the NOx emissions values
egu_df['nox_mass (lbs)'] = ml_nox.loc[idx, ml_nox.columns[5:]].values
# Replace the load values
egu_df['gload (MW-hr)'] = ed_gen.loc[idx, ed_gen.columns[5:]].values
# Combine this new unit data back into the base_df
base_df.update(egu_df)

Check to make sure that the overwrite worked

In [62]:
# tmp = base_df.loc[(base_df['orispl_code'] == egu_orispl) & (base_df['unitid'] == egu_unitid)]
# tmp = tmp.loc[base_df['datetime'].isin(ml_co2.columns[5:])]
# tmp

In [63]:
# This plot should be blank
# (tmp['co2_mass (tons)'] - ml_co2.loc[idx, ml_co2.columns[5:]]).plot()

Here's the algorithm in a loop

In [48]:
# Get the name of an individual EGU -- this is how units are identified in the NY Simple Net & the ML
for idx in ml_co2.index:
    # Get the ORISPL and the Unit ID
    egu_orispl = ml_co2.loc[idx].ORISPL
    egu_unitid = ml_co2.loc[idx]['Unit ID']
    print(f'Working on ORISPL: {egu_orispl}\tUNIT ID:{egu_unitid}')

    # Extract this ORISPL and UNIT ID from the base DataFrame
    egu_df = base_df.loc[(base_df['orispl_code'] == egu_orispl) & (base_df['unitid'] == egu_unitid)]
    # Extract the correct time window
    egu_df = egu_df.loc[base_df['datetime'].isin(ml_co2.columns[5:])]
    if len(egu_df) == 0:
        print('Warning: this unit was not found in the CAMD data... skipping')
    else:
        # Replace the CO2 emissions values
        # NOTE: this is probably a dangerous way of doing this -- might be better to add the datetime as another index in the egu_df
        egu_df['co2_mass (tons)'] = ml_co2.loc[idx, ml_co2.columns[5:]].values
        # Replace the SO2 emissions values
        egu_df['so2_mass (lbs)'] = ml_so2.loc[idx, ml_so2.columns[5:]].values
        # Replace the NOx emissions values
        egu_df['nox_mass (lbs)'] = ml_nox.loc[idx, ml_nox.columns[5:]].values
        # Replace the load values
        egu_df['gload (MW-hr)'] = ed_gen.loc[idx, ed_gen.columns[5:]].values
        # Combine this new unit data back into the base_df
        base_df.update(egu_df)

Working on ORISPL: 7910	UNIT ID:2301
Working on ORISPL: 7910	UNIT ID:2302
Working on ORISPL: 10619	UNIT ID:1
Working on ORISPL: 2490	UNIT ID:20
Working on ORISPL: 2490	UNIT ID:30
Working on ORISPL: 55375	UNIT ID:CT1
Working on ORISPL: 55375	UNIT ID:CT2
Working on ORISPL: 55375	UNIT ID:CT3
Working on ORISPL: 55375	UNIT ID:CT4
Working on ORISPL: 55243	UNIT ID:CT2-1A
Working on ORISPL: 55243	UNIT ID:CT2-1B
Working on ORISPL: 55243	UNIT ID:CT2-2A
Working on ORISPL: 55243	UNIT ID:CT2-2B
Working on ORISPL: 55243	UNIT ID:CT2-3A
Working on ORISPL: 55243	UNIT ID:CT2-3B
Working on ORISPL: 55243	UNIT ID:CT2-4A
Working on ORISPL: 55243	UNIT ID:CT2-4B
Working on ORISPL: 55243	UNIT ID:CT3-1A
Working on ORISPL: 55243	UNIT ID:CT3-1B
Working on ORISPL: 55243	UNIT ID:CT3-2A
Working on ORISPL: 55243	UNIT ID:CT3-2B
Working on ORISPL: 55243	UNIT ID:CT3-3A
Working on ORISPL: 55243	UNIT ID:CT3-3B
Working on ORISPL: 55243	UNIT ID:CT3-4A
Working on ORISPL: 55243	UNIT ID:CT3-4B
Working on ORISPL: 55243	UNIT ID:

In [49]:
base_df.head()

,ertac_region,ertac_fuel_unit_type_bin,state,facility_name,orispl_code,unitid,op_date,op_hour,op_time,gload (MW-hr),...,nox_rate (lbs/mmBtu),nox_rate_measure_flg,nox_mass (lbs),nox_mass_measure_flg,co2_mass (tons),co2_mass_measure_flg,co2_rate (tons/mmBtu),co2_rate_measure_flg,heat_input (mmBtu),datetime
0,SRSE,Boiler Gas,AL,Barry,3,1,2016-01-01,00,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-01-01 00:00:00
1,SRSE,Boiler Gas,AL,Barry,3,1,2016-01-01,01,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-01-01 01:00:00
2,SRSE,Boiler Gas,AL,Barry,3,1,2016-01-01,02,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-01-01 02:00:00
3,SRSE,Boiler Gas,AL,Barry,3,1,2016-01-01,03,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-01-01 03:00:00
4,SRSE,Boiler Gas,AL,Barry,3,1,2016-01-01,04,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-01-01 04:00:00


In [ ]:
# Save the updated dataset to a new CSV
base_df = base_df.drop(columns=['datetime'])
base_df.to_csv('updated_test_calc_hourly_base.csv', index=False)

In [ ]:
# Now, test the prepemis.update_camd() function
in_emis_file = 'test_calc_hourly_base.csv'
co2_file = '../ny_emis/ml_output/pred_xg_co2.csv'
nox_file = '../ny_emis/ml_output/pred_xg_nox.csv'
so2_file = '../ny_emis/ml_output/pred_xg_so2.csv'
gen_file = '../ny_emis/ed_output/thermal_with_renewable_20160805_20160815.csv'
lu_file = '../ny_emis/ed_output/RGGI_to_NYISO.csv'
out_emis_file = 'updated_test_calc_hourly_base_v2.csv'

prepemis.update_camd(in_emis_file=in_emis_file, co2_file=co2_file, 
                     nox_file=nox_file, so2_file=so2_file, 
                     gen_file=gen_file, lu_file=lu_file, 
                     out_emis_file=out_emis_file)